In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False
rc('font', family='Malgun Gothic')
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [55]:
#!pip install threadpoolctl==3.1.0

  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0


In [57]:
#!pip install numpy==1.21.4

                                              0.0/14.0 MB ? eta -:--:--
     -                                        0.4/14.0 MB 8.7 MB/s eta 0:00:02
     --                                       0.9/14.0 MB 9.7 MB/s eta 0:00:02
     ----                                     1.5/14.0 MB 10.7 MB/s eta 0:00:02
     -----                                    2.0/14.0 MB 11.5 MB/s eta 0:00:02
     -------                                  2.5/14.0 MB 11.6 MB/s eta 0:00:01
     ---------                                3.2/14.0 MB 11.4 MB/s eta 0:00:01
     ----------                               3.8/14.0 MB 11.4 MB/s eta 0:00:01
     ------------                             4.4/14.0 MB 11.1 MB/s eta 0:00:01
     -------------                            4.9/14.0 MB 11.1 MB/s eta 0:00:01
     ---------------                          5.4/14.0 MB 11.1 MB/s eta 0:00:01
     ----------------                         5.9/14.0 MB 10.8 MB/s eta 0:00:01
     ------------------                       6.4/

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\xswer\\miniconda3\\envs\\ds_study\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [3]:
url_raw = "https://raw.githubusercontent.com/DataResolvere/Project/main/Dacon_Crime/train.csv"
url_test = "https://raw.githubusercontent.com/DataResolvere/Project/main/Dacon_Crime/test.csv"

raw_df = pd.read_csv(url_raw)
raw_df.drop(columns="ID", inplace=True)

test_df = pd.read_csv(url_test)
test_df.drop(columns="ID", inplace=True)

In [4]:
raw_df

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,9,화요일,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,11,화요일,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,8,일요일,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,5,월요일,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,9,일요일,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,4,일요일,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,1
84402,8,목요일,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,차도,0
84403,7,일요일,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
84404,1,화요일,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,주거지,1


In [5]:
raw_df.drop_duplicates(keep='first', inplace=True)
raw_df

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,9,화요일,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,11,화요일,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,8,일요일,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,5,월요일,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,9,일요일,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,4,일요일,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,1
84402,8,목요일,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,차도,0
84403,7,일요일,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
84404,1,화요일,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,주거지,1


In [6]:
make_en = LabelEncoder()


raw_df["요일"] = make_en.fit_transform(raw_df["요일"])
test_df["요일"] = make_en.transform(test_df["요일"])

raw_df["범죄발생지"] = make_en.fit_transform(raw_df["범죄발생지"])
test_df["범죄발생지"] = make_en.transform(test_df["범죄발생지"])

In [7]:
raw_df["안개"] = raw_df["안개"].astype("int64")
raw_df["짙은안개"] = raw_df["짙은안개"].astype("int64")
raw_df["번개"] = raw_df["번개"].astype("int64")
raw_df["진눈깨비"] = raw_df["진눈깨비"].astype("int64")
raw_df["연기/연무"] = raw_df["연기/연무"].astype("int64")
raw_df["눈날림"] = raw_df["눈날림"].astype("int64")
raw_df["서리"] = raw_df["서리"].astype("int64")

test_df["안개"] = test_df["안개"].astype("int64")
test_df["짙은안개"] = test_df["짙은안개"].astype("int64")
test_df["번개"] = test_df["번개"].astype("int64")
test_df["진눈깨비"] = test_df["진눈깨비"].astype("int64")
test_df["연기/연무"] = test_df["연기/연무"].astype("int64")
test_df["눈날림"] = test_df["눈날림"].astype("int64")
test_df["서리"] = test_df["서리"].astype("int64")

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84364 entries, 0 to 84405
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   월        84364 non-null  int64  
 1   요일       84364 non-null  int32  
 2   시간       84364 non-null  int64  
 3   소관경찰서    84364 non-null  int64  
 4   소관지역     84364 non-null  float64
 5   사건발생거리   84364 non-null  float64
 6   강수량(mm)  84364 non-null  float64
 7   강설량(mm)  84364 non-null  float64
 8   적설량(cm)  84364 non-null  float64
 9   풍향       84364 non-null  float64
 10  안개       84364 non-null  int64  
 11  짙은안개     84364 non-null  int64  
 12  번개       84364 non-null  int64  
 13  진눈깨비     84364 non-null  int64  
 14  서리       84364 non-null  int64  
 15  연기/연무    84364 non-null  int64  
 16  눈날림      84364 non-null  int64  
 17  범죄발생지    84364 non-null  int32  
 18  TARGET   84364 non-null  int64  
dtypes: float64(6), int32(2), int64(11)
memory usage: 12.2 MB


In [9]:
raw_df[['소관경찰서', '소관지역', '범죄발생지']]

,소관경찰서,소관지역,범죄발생지
0,137,8.0,10
1,438,13.0,10
2,1729,47.0,6
3,2337,53.0,7
4,1439,41.0,8
...,...,...,...
84401,336,11.0,10
84402,2149,38.0,10
84403,29,46.0,10
84404,536,25.0,7


In [10]:
# Clustering
km = KMeans(n_clusters=20, random_state=13)
km_data = km.fit(raw_df[['안개', '짙은안개', '번개', '진눈깨비','서리','눈날림', '연기/연무']])
raw_df['clustering'] = km_data.labels_
test_df['clustering'] = km_data.predict(test_df[['안개', '짙은안개', '번개', '진눈깨비','서리','눈날림', '연기/연무']])


km_data = km.fit(raw_df[['소관경찰서', '소관지역', '범죄발생지']])
raw_df['clustering2'] = km_data.labels_
test_df["clustering2"] = km_data.predict(test_df[['소관경찰서', '소관지역', '범죄발생지']])

C:\Users\xswer\miniconda3\envs\ds_study\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\xswer\miniconda3\envs\ds_study\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
raw_df = raw_df.drop(columns=['안개', '짙은안개', '번개', '진눈깨비','서리','눈날림', '강설량(mm)', '적설량(cm)', "연기/연무"])
test_df = test_df.drop(columns=['안개', '짙은안개', '번개', '진눈깨비','서리','눈날림', '강설량(mm)', '적설량(cm)', "연기/연무"])

In [13]:
x = raw_df.drop(columns='TARGET')
y = raw_df[['TARGET']]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=13)

In [15]:
cat = CatBoostClassifier(n_estimators=3000, max_depth=10, random_seed=1000, learning_rate=0.04, bootstrap_type ='MVS')
cat.fit(x_train, y_train,
eval_set=[(x_train, y_train), (x_test, y_test)],
early_stopping_rounds=50, cat_features=["월", "요일", "시간", 'clustering', '범죄발생지', 'clustering2'],
verbose=100)

pred = cat.predict(x_test)
print(accuracy_score(y_test, pred))
cat_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True)).transpose()

0:	learn: 1.0889030	test: 1.0888741	test1: 1.0889360	best: 1.0889360 (0)	total: 394ms	remaining: 19m 42s
100:	learn: 0.9460979	test: 0.9528923	test1: 0.9626230	best: 0.9626230 (100)	total: 30.2s	remaining: 14m 27s
200:	learn: 0.9291362	test: 0.9461449	test1: 0.9608953	best: 0.9608929 (198)	total: 1m 10s	remaining: 16m 26s
300:	learn: 0.9119291	test: 0.9400794	test1: 0.9599505	best: 0.9599505 (300)	total: 2m 8s	remaining: 19m 13s
400:	learn: 0.8887595	test: 0.9293012	test1: 0.9588371	best: 0.9588211 (397)	total: 3m 37s	remaining: 23m 27s
500:	learn: 0.8685727	test: 0.9208680	test1: 0.9586184	best: 0.9585204 (494)	total: 5m 24s	remaining: 26m 57s
600:	learn: 0.8477886	test: 0.9133522	test1: 0.9584238	best: 0.9583388 (594)	total: 6m 33s	remaining: 26m 9s
700:	learn: 0.8298368	test: 0.9070488	test1: 0.9582283	best: 0.9581506 (690)	total: 7m 53s	remaining: 25m 51s
800:	learn: 0.8128462	test: 0.9013844	test1: 0.9581902	best: 0.9579765 (758)	total: 9m 5s	remaining: 24m 57s
Stopped by overfitt